# Elektrooptični pojav

In [39]:
#Copyright (c) 2020 Tomaž Mertelj

%matplotlib inline
from ipywidgets import interact,interactive_output, HBox, VBox, Button, Output,Valid
from ipywidgets import FloatSlider, Select, Dropdown, Layout,Label,HTML, Text, FloatLogSlider
import matplotlib.pyplot as plt
import numpy as np
import math

tau=0.004
noise=0.003
fmine=0
fmaxe=3
fstepe=0.05

def generate(ampl,freq,pol_angle,trange,sens,tc,fs_flag):
    pol_angle=pol_angle+30
    t=np.arange(0,trange,trange/500)
    Uref=ampl*np.exp(2j*math.pi*freq*t)
    Upsi=np.sin(2*(pol_angle/90*math.pi+(Uref/(1+2j*math.pi*freq*tau)).real))
    tlock=np.arange(0,2*math.pi,2*math.pi/100)
    Urlock=np.exp(1j*tlock)
    Uplock=np.sin(2*(pol_angle/90*math.pi+(ampl*Urlock/(1+2j*math.pi*freq*tau)).real))
    alock=np.dot(Uplock,Urlock)/50
    x_widget.value='{:.3}'.format(alock.real+np.random.normal(0,noise/tc))
    y_widget.value='{:.3}'.format(alock.imag+np.random.normal(0,noise/tc))
    fig, axes = plt.subplots(2,1)
    axes[0].plot(t, Uref.real)
    axes[1].plot(t, Upsi)
    axes[1].set_ylim([-sens,sens])
    axes[0].set_ylabel(r'$U_{ref} (V)$', fontsize=20)
    axes[1].set_ylabel(r'$I_{p}$', fontsize=20)
    axes[1].set_xlabel(r'$t$ (s)', fontsize=20)    
    axes[0].set_xlabel(r'$t$ (s)', fontsize=20)
    data_widget.clear_output()
    if fs_flag:
        fs_flag_widget.value=False
        with data_widget:
            print('frekvenca\tX\tY')
        fr=10**np.arange(fmine,fmaxe,fstepe)
        for freq in fr:
            Uplock=np.sin(2*(pol_angle/90*math.pi+(ampl*Urlock/(1+2j*math.pi*freq*tau)).real))
            alock=np.dot(Uplock,Urlock)/50
            with data_widget:
                print(f'{freq:.3}\t{alock.real+np.random.normal(0,noise/tc):.3}\t{alock.imag+np.random.normal(0,noise/tc):.3}')
    
    
freq_widget = FloatLogSlider(min=fmine, max=fmaxe,step=fstepe, value=10, base=10, 
                          continuous_update=False,description='Frekvenca (Hz)',
                             style={'description_width': '30%'})
ampl_widget = FloatSlider(min=0, max=2, step=0.05, value=1, 
                          continuous_update=False,description='Amplituda $U_{ref}$ (V)',
                          style={'description_width': '40%'})
pol_angle_widget = FloatSlider(min=0, max=360, step=1, value=0, 
                               continuous_update=False,description='Kot anal. (st.)',
                               style={'description_width': '30%'})
trange_widget =  Dropdown(
    options=[0.01,0.03,.1,.3,1,3],
    value=1,
    description='Horiz. range (s):',
    disabled=False,
    layout=Layout(width='60%', height='auto'),
    style={'description_width': '60%'}
)

sens_widget = Dropdown(
    options=[0.01,0.03,0.1,0.3,1,3],
    value=1,
    description='Vert. range (V):',
    disabled=False,
    layout=Layout(width='60%', height='auto'),
    style={'description_width': '60%'}
)

tc_widget = Dropdown(
    options=[0.01,0.03,0.1,0.3,1,3],
    value=1,
    description='Časovna konstanta (s):',
    disabled=False,
#    layout=Layout(width='60%', height='auto'),
    style={'description_width': '70%'}
)

x_widget=Text(
    value='',
    description='X (V):',
    disabled=True,
    style={'description_width': '70%'}
)

y_widget=Text(
    value='',
    description='Y (V):',
    disabled=True,
    style={'description_width': '70%'}
)

data_widget=Output(layout={'border': '1px solid black'},description='Meritev:')

fs_flag_widget=Valid(
    value=False,
    description='',
)

fs_widget=Button(
    description='Frekvenčni prelet',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generira frekvenčno odvisnost.',
    icon=''
)

def on_fs_widget_clicked(b):
    fs_flag_widget.value=True
    fs_widget.enabled=False


fs_widget.on_click(on_fs_widget_clicked)


out=interactive_output(generate, {'ampl':ampl_widget,
                                  'freq':freq_widget, 
                                  'pol_angle':pol_angle_widget, 
                                  'trange':trange_widget,
                                  'sens':sens_widget,
                                  'tc':tc_widget,
                                  'fs_flag':fs_flag_widget
                                 });

params_box=VBox([HTML('<b> Parametri poskusa:</>'),pol_angle_widget])

lockin_box=VBox([HTML('<b> Fazno občutljiv ojačevalnik:</>'),freq_widget, ampl_widget,tc_widget,x_widget,y_widget,fs_widget])

oscillo_box=VBox([HTML('<b> Osciloskop:</>',layout=Layout(width='20%')),
                  HBox([VBox([trange_widget,sens_widget],
                             layout=Layout(width='30%', height='auto')),
                        out])
                 ])

data_box=VBox([HTML('<b>Rezultati:</>'),data_widget])

display(HBox([params_box,HTML('',layout=Layout(width='10%')),lockin_box]), oscillo_box,data_box)